## Trivial Mean and Moving Average Benchmarks

Kaggle [Kernels](https://www.kaggle.com/kernels) and Kaggle [Forums](https://www.kaggle.com/forums) provides the users of Kaggle Data Science Competitions with excellent resources to share scripts, benchmarks and information regarding Data Science. The idea behind the notebook is to try to get the ENGIE DSC community to share ideas and code.

Unfortunately, I am not familiar with R so if someone else has a similar script that they could post for the benifit of R users then this would be great.

In [9]:
import os
import re
import sys
import itertools

import numpy as np
import pandas as pd
import seaborn as sns
import datetime as dt
import missingno as mn
import matplotlib.pylab as plt

from bokeh import mpl
from bokeh.io import hplot
from bokeh.models import *
from bokeh.plotting import *
from IPython.display import display, Image
from sklearn.metrics import mean_squared_error as mse

sys.path.append(r'/home/josh/Documents/Python Scripts/Packages')

import Call_All_Functions as caf

%matplotlib inline

plt.rcParams['figure.figsize'] = 25, 5

In [10]:
path = r'/home/josh/Documents/Python Scripts/Data Science Challenges/ENGIE DSC/Supply Demand Forecast/'
raw = 'Raw Data/'

In [141]:
def add_date(df):
    """
    A simple wrapper for pd.datetime() to help up turning a date string column 
    into a datetime column. The columns are identified as datetime columns if
    their name contains the words "date" or "Date".
    
    Parameters
    ----------
    df : Pandas DataFrame
    
    Warning: Can be slow for the less conventional date formats due to the
    infer_datetime_format option in pd.to_datetime().
    
    Examples
    --------
    >>> df_datetime = add_date(df1)

    Returns
    -------
    df : Pandas DataFrame
        Pandas DataFrame (with datetime columns)
    """
    for col in df.columns:
        if re.match('.*[Dd]ate.*', col):
            df.ix[:, col] = pd.to_datetime(df.ix[:, col])
    return df


def set_custom_index(df, custom_index=None):
    """
    A simple wrapper for dataframe.set_index() to help with setting
    DataFrame indexes from a list of potential target columns.
    
    Parameters
    ----------
    df : Pandas DataFrame
    custom_index : list, default None
        A list of desired DataFrame index options. These should be DataFrame
        column names. If more than one is matched the latter in the list is
        overrides the any former. When left as None the list defaults to
        ['application_date', 'from_date', 'Date', 'date_hour'].
        
    Examples
    --------
    >>> indexed_df = set_custom_index(df1)
    >>> custom_indexed_df = set_custom_index(df1, custom_index=['A', 'B', 'C'])
    
    Returns
    -------
    df : Pandas DataFrame
        Indexed Pandas DataFrame
    """
    
    if not custom_index:
        custom_index = ['application_date', 'from_date', 'Date', 'date_hour']
    
    for col in df.columns:
        if col in custom_index:
            df = df.set_index(col)
    return df

In [139]:
data = pd.read_csv(path + raw + 'train.CSV')
data = add_date(data)
data = set_custom_index(data)

files = ['balance delta with IGCC.CSV',
         'Intraday.CSV',
         'ID_capacity.CSV',
         'Imbalance_quantity_Germany.CSV',
         'imbalance.CSV',
         'price_ladder.CSV',
         'wind_forecast.CSV',
         'Day Ahead Prices.CSV',
         'Intraday_outages_nl.CSV',
         'ID_avPrice_per_Interval.CSV',
         'Imbalance_quantity_Belgium.CSV']

df = pd.read_csv(path + raw + 'Day Ahead Prices.CSV')

In [61]:
data = data.reindex(pd.date_range(data.index[0], data.index[-1], freq='H'))

submissions = '/Submissions/Exploratory Submissions/'

#### Use last hour's Read

In [180]:
data.fillna(method='ffill').iloc[4::5, :].to_csv(path + submissions + 'Use_last_read.csv', index=True, index_label='date_hour')

print('RMSE = %.3f' % 43.8924283825)

RMSE = 43.892


#### Mean

In [151]:
data.fillna(data.mean()).iloc[4::5, :].to_csv(path + submissions + 'Use_mean.csv', index=True, index_label='date_hour')

print('RMSE = %.3f' % 43.5395866019)

RMSE = 43.540


#### Use Mean

In [153]:
data.fillna(data.rolling(window=data.shape[0], min_periods=1).mean()
           ).iloc[4::5, :].to_csv(path + submissions + 'Use_rolling_mean_full_window.csv', index=True, index_label='date_hour')

print('RMSE = %.3f' % 43.5510819387)

RMSE = 43.551


#### Rolling 2-week-window Mean

In [184]:
data.fillna(data.rolling(window=2*7*24, min_periods=1).mean()
           ).iloc[4::5, :].to_csv(path + submissions + 'Use_rolling_mean_2_week_window.csv', index=True, index_label='date_hour')

print('RMSE = %.3f' % 42.8401553102)

RMSE = 42.840


#### EWMA with defined parameters

In [229]:
alpha = 0.5
for alpha in np.linspace(0.0001, 1, 15):
    print('Alpha: %.3f, RMSE: %.3f' % (alpha, (np.sqrt(mse(data.ewm(alpha=alpha).mean().imbalance_price_pos, data.fillna(0).imbalance_price_pos))
                                                + np.sqrt(mse(data.ewm(alpha=alpha).mean().imbalance_price_pos, data.fillna(0).imbalance_price_pos)))/2))
    
# Use alpha = 0.84
data.fillna(data.ewm(alpha=0.84).mean(
    )).iloc[4::5, :].to_csv(path + submissions + 'Use_ewma_alpha_0.84.csv', index=True, index_label='date_hour')

print('\nRMSE = %.3f' % 41.5449986952)

Alpha: 0.000, RMSE: 41.309
Alpha: 0.072, RMSE: 37.079
Alpha: 0.143, RMSE: 34.493
Alpha: 0.214, RMSE: 32.295
Alpha: 0.286, RMSE: 30.380
Alpha: 0.357, RMSE: 28.713
Alpha: 0.429, RMSE: 27.284
Alpha: 0.500, RMSE: 26.098
Alpha: 0.571, RMSE: 25.158
Alpha: 0.643, RMSE: 24.462
Alpha: 0.714, RMSE: 24.001
Alpha: 0.786, RMSE: 23.760
Alpha: 0.857, RMSE: 23.726
Alpha: 0.929, RMSE: 23.902
Alpha: 1.000, RMSE: 24.305

RMSE = 41.545


#### Rolling Groupby

In [121]:
data.groupby([data.index.dayofweek, data.index.hour]
            ).rolling(window=4, min_periods=1).mean().reset_index(level=[0, 1], drop=True
                                                                 ).fillna(value=data.rolling(window=168, min_periods=1).mean()
                                                                          ).sort_index().iloc[4::5, :].to_csv(path + submissions + 
                                                                                                              'Use_rolling_groupby_dotw_hour.csv',
                                                                                                               index=True,
                                                                                                               index_label='date_hour')

print('\nRMSE = %.3f' % 49.2823456622)


RMSE = 49.282


#### Groupby Hour and Day of the Week EWMA

In [123]:
data.groupby([data.index.dayofweek, data.index.hour]
            ).apply(lambda x: x.ewm(alpha=0.84).mean()).fillna(value=data.rolling(window=168, min_periods=1).mean()
                                                            ).sort_index().iloc[4::5, :].to_csv(path + submissions + 
                                                                                                              'Use_ewma_groupby_dotw_hour.csv',
                                                                                                               index=True,
                                                                                                               index_label='date_hour')

print('\nRMSE = %.3f' % 54.0510640811)


RMSE = 54.051


#### Groupby Hour with EWMA

In [2]:
data.groupby([data.index.hour]
            ).apply(lambda x: x.ewm(alpha=0.84).mean()).fillna(value=data.rolling(window=168, min_periods=1).mean()
                                                            ).sort_index().iloc[4::5, :].to_csv(path + submissions + 
                                                                                                              'Use_ewma_groupby_hour.csv',
                                                                                                               index=True,
                                                                                                               index_label='date_hour')

print('\nRMSE = %.3f' % 54.7180950271)


RMSE = 54.718


#### Groupby Day of the Week with EWMA

In [3]:
data.groupby([data.index.dayofweek]
            ).apply(lambda x: x.ewm(alpha=0.84).mean()).fillna(value=data.rolling(window=168, min_periods=1).mean()
                                                            ).sort_index().iloc[4::5, :].to_csv(path + submissions + 
                                                                                                              'Use_ewma_groupby_dotw.csv',
                                                                                                               index=True,
                                                                                                               index_label='date_hour')

print('\nRMSE = %.3f' % 41.4744459964)


RMSE = 41.474
